# Lab 3: Word Embeddings and Language Modelling

Adam Ek

In this lab we'll explore constructing *static* word embeddings (i.e. word2vec) and building language models. We'll also evaluate these systems on intermediate tasks, namely word similarity and identifying "good" and "bad" sentences.

* For this we'll use pytorch. Some basic operations that will be useful can be found here: https://jhui.github.io/2018/02/09/PyTorch-Basic-operations
* In general: we are not interested in getting state-of-the-art performance :) focus on the implementation and not results of your model. For this reason, you can use a subset of the dataset: the first 5000-10 000 sentences or so, on linux/mac: ```head -n 10000 inputfile > outputfile```. 
* If possible, use the MLTGpu, it will make everything faster :)

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# for gpu, replace "cpu" with "cuda:n" where n is the index of the GPU
device = torch.device('cpu')
torch.cuda.is_available()

False

# Word2Vec embeddings

In this first part we'll construct a word2vec model which will give us *static* word embeddings (that is, they are fixed after training).

After we've trained our model we will evaluate the embeddings obtained on a word similarity task.

## Formatting data


First we need to load some data, you can download the file on Canvas under files/assignments/03-lab-data/wiki-corpus.50000.txt. The file contains 50 000 sentences randomly selected from the complete wikipedia. Each line in the file contains one sentence. The sentences are whitespace tokenized.

Your first task is to create a dataset suitable for word2vec. That is, we define some ```window_size``` then iterate over all sentences in the dataset, putting the center word in one field and the context words in another (separate the fields with ```tab```).

For example, the sentece "this is a lab" with ```window size = 4``` will be formatted as:
```
center, context
---------------------
this    is a
is      this a lab
a       this is lab
lab     is a
```

this will be our training examples when training the word2vec model.

[3 marks]

In [57]:
import math

data_path = 'data/'
WINDOW_SIZE = 4
OCCURRENCE_THRESHOLD = 1
UNKNOWN_TOKEN = '<UNK>'


def corpus_reader(data_path):
    before = math.floor(WINDOW_SIZE / 2)
    after = WINDOW_SIZE - before

    samples = []
    with open(data_path, encoding='UTF-8') as f:
        lines = f.readlines()[:10]
        occurrences = {}
        for line in lines:
            tokenized = [word.lower() for word in line.rstrip('\n').split(' ')]
            for word in tokenized:
                occurrences[word] = occurrences.get(word, 0) + 1

        unknown_words = [word for word in occurrences if occurrences[word] <= OCCURRENCE_THRESHOLD]

        for line_index, line in enumerate(lines):
            tokenized = [UNKNOWN_TOKEN if word in unknown_words else word.lower() for word in line.rstrip('\n').split(' ')]
            for index, word in enumerate(tokenized):
                if word != UNKNOWN_TOKEN:
                    context = []
                    for i in range(index - before, index):
                        if i >= 0:
                            context.append(tokenized[i])
                    for i in range(index + 1, index + after + 1):
                        if i < len(tokenized):
                            context.append(tokenized[i])
                    samples.append((word, context))
    return samples

In [58]:
#print(corpus_reader(data_path + 'wiki-corpus.50000.txt')[:1000])

We sampled 50 000 senteces completely random from the *whole* wikipedia for our training data. Give some reasons why this is good, and why it might be bad. (*note*: We'll have a few questions like these, one or two reasons for and against is sufficient)

[2 marks]

### Good:
- different subjects
- big vocabulary
- words are used in 'true' meaning (not metaphorical) grammatically because of informative nature

### Bad:
- one type of language
- informative
- may lose variety that can be in different kinds of texts
- depending on goal

### Loading the data

We now need to load the data in an appropriate format for torchtext (https://torchtext.readthedocs.io/en/latest/). We'll use PyText for this and it'll follow the same structure as I showed you in the lecture (remember to lower-case all tokens). Create a function which returns a (bucket)iterator of the training data, and the vocabulary object (```Field```). 

(*hint1*: you can format the data such that the center word always is first, then you only need to use one field)

(*hint2*: the code I showed you during the leture is available in /files/pytorch_tutorial/ on canvas)

[4 marks]

In [59]:
from collections import namedtuple
from torch.utils.data import Dataset


class CustDataset(Dataset):
    def __init__(self, data):
        self.vocab = {}
        for target_word, context in data:
            if target_word not in self.vocab:
                self.vocab[target_word] = len(self.vocab)
            for word in context:
                if word not in self.vocab:
                    self.vocab[word] = len(self.vocab)

        self.data = data

    def __getitem__(self, idx):
        target_word, context = self.data[idx]

        encoded_context = [self.vocab[word] for word in context]
        encoded_context.extend([len(encoded_context)] * (WINDOW_SIZE - len(encoded_context)))

        Sample = namedtuple('Sample', 'target_word context')

        return Sample(self.vocab[target_word], encoded_context)

    def __len__(self):
        return len(self.data)

    def getVocab(self):
        return self.vocab

In [60]:
from torch.utils.data import DataLoader


def get_data(path, batch_size):
    samples = corpus_reader(path)
    dataset = CustDataset(samples)
    dataloader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            collate_fn=lambda x: x)

    return dataloader

We lower-cased all tokens above; give some reasons why this is a good idea, and why it may be harmful to our embeddings.

[2 marks]

### Good
- beginning of sentences
- no duplicates

### Bad
- some proper nouns may be missed, those that may have a counterpart in a common noun and vice versa (e.g. Mark/mark)
- can create problems in other languages (e.g. German)


## Word Embeddings Model

We will implement the CBOW model for constructing word embedding models.

In [61]:
import torch.optim as optim

In the CBOW model we try to predict the center word based on the context. That is, we take as input ```n``` context words, encode them as vectors, then combine them by summation. This will give us one embedding. We then use this embedding to predict *which* word in our vocabuary is the most likely center word. 

Implement this model 

[7 marks]

In [62]:
class CBOWModel(nn.Module):
    def __init__(self, vocab_size, dimensions):
        super(CBOWModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, dimensions)
        self.prediction = nn.Linear(dimensions, vocab_size)

    def forward(self, context):
        embedded_context = self.embeddings(context)
        projection = self.projection_function(embedded_context)
        predictions = self.prediction(projection)

        return predictions

    def projection_function(self, xs):
        """
        This function will take as input a tensor of size (B, S, D)
        where B is the batch_size, S the window size, and D the dimensionality of embeddings
        this function should compute the sum over the embedding dimensions of the input, 
        that is, we transform (B, S, D) to (B, 1, D) or (B, D) 
        """
        xs_sum = torch.sum(xs, dim=1)
        return xs_sum

Now we need to train the models. First we define which hyperparameters to use. (You can change these, for example when *developing* your model you can use a batch size of 2 and a very low dimensionality (say 10), just to speed things up). When actually training your model *fo real*, you can use a batch size of [8,16,32,64], and embedding dimensionality of [128,256].

In [63]:
# you can change these numbers to suit your needs :)
word_embeddings_hyperparameters = {'epochs': 3,
                                   'batch_size': 8,
                                   'embedding_size': 128,
                                   'learning_rate': 0.001}

In [64]:
dataloader = get_data(data_path + 'wiki-corpus.50000.txt', word_embeddings_hyperparameters['batch_size'])

In [65]:
cbow_model = CBOWModel(len(dataloader.dataset), word_embeddings_hyperparameters['embedding_size'])
torch.cuda.empty_cache()
cbow_model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cbow_model.parameters(), lr=word_embeddings_hyperparameters['learning_rate'])

Train your model. Iterate over the dataset, get outputs from your model, calculate loss and backpropagate.

We mentioned in the lecture that we use Negative Log Likelihood (https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html) loss to train Word2Vec model. In this lab we'll take a shortcut when *training* and use Cross Entropy Loss (https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), basically it combines ```log_softmax``` and ```NLLLoss```. So what your model should output is a *score* for each word in our vocabulary. The ```CrossEntropyLoss``` will then assign probabilities and calculate the negative log likelihood loss.

[3 marks]

In [66]:
%%time

# start training loop
for epoch in range(word_embeddings_hyperparameters['epochs']):
    total_loss = 0
    print(f'batch count: {math.floor(len(dataloader.dataset) / dataloader.batch_size)}')
    for i, batch in enumerate(dataloader):
        torch.cuda.empty_cache()
        contexts = torch.tensor([sample.context for sample in batch])
        target_words = torch.tensor([sample.target_word for sample in batch])

        contexts = contexts.to(device)
        target_words = target_words.to(device)

        # send your batch of sentences to the model
        output = cbow_model(contexts)

        # compute the loss, you'll need to reshape the input
        # you can read more about this is the documentation for
        # CrossEntropyLoss
        loss = loss_fn(output.view(-1, len(dataloader.dataset)), target_words.view(-1))
        total_loss += loss.item()

        # print average loss for the epoch
        print(f'epoch {epoch},', f'batch {i}:', np.round(total_loss / (i + 1), 4), end='\r')

        # compute gradients
        loss.backward()

        # update parameters
        optimizer.step()

        # reset gradients
        optimizer.zero_grad()
    print()


batch count: 21
epoch 0, batch 21: 5.5151
batch count: 21
epoch 1, batch 21: 4.0107
batch count: 21
epoch 2, batch 21: 3.3716
CPU times: user 423 ms, sys: 9.7 ms, total: 432 ms
Wall time: 262 ms


## Evaluating the model

In [67]:
# your code goes here

def read_wordsim(path, vocab, embeddings):
    count = 0
    dataset_sims = []
    model_sims = []
    with open(path) as f:
        for line in f:
            word1, word2, score = line.split()

            score = float(score)

            # get the index for the word
            if word1 in vocab and word2 in vocab:
                count += 1
                print(count, end='\r')
                dataset_sims.append(score)
                word1_idx = vocab[word1]
                word2_idx = vocab[word2]

                # get the embedding of the word
                word1_emb = embeddings.weight[word1_idx]
                word2_emb = embeddings.weight[word2_idx]
                # compute cosine similarity, we'll use the version included in pytorch functional
                # https://pytorch.org/docs/master/generated/torch.nn.functional.cosine_similarity.html
                cosine_similarity = F.cosine_similarity(word1_emb, word2_emb, dim=0)

                model_sims.append(cosine_similarity.item())

    return dataset_sims, model_sims


path = 'wordsim_similarity_goldstandard.txt'
data, model = read_wordsim(data_path + path, dataloader.dataset.getVocab(), cbow_model.embeddings)
pearson_correlation = np.corrcoef(data, model)

# the non-diagonals give the pearson correlation,
print(pearson_correlation)

[[nan nan]
 [nan nan]]


/home/dominik/anaconda3/envs/computational-semantics/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/dominik/anaconda3/envs/computational-semantics/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/home/dominik/anaconda3/envs/computational-semantics/lib/python3.9/site-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/dominik/anaconda3/envs/computational-semantics/lib/python3.9/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/home/dominik/anaconda3/envs/computational-semantics/lib/python3.9/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


In [68]:
def test():
    words = set()
    with open('data/wordsim_similarity_goldstandard.txt') as f:
        for line in f:
            words_gold = line.split('\t')
            words.add(words_gold[0])
            words.add(words_gold[1])

    words_wiki = set()
    with open('data/wiki-corpus.50000.txt') as f:
        for line in f:
            words_tmp = line.split(' ')
            words_wiki.update(words_tmp)


    for word in words:
        if word not in words_wiki:
            print(word)
    print(len(words))

test()

yen
carnivore
stove
graveyard
madhouse
brandy
rook
jaguar
recess
thunderstorm
gem
feline
buck
277


### Results
#### full wiki-50000
```
Epoch 0: 7.5331
Epoch 1: 7.1656
Epoch 2: 7.0064
CPU times: user 11h 28min 53s, sys: 6h 9min 52s, total: 17h 38min 45s
Wall time: 17h 46min 41s

[[1.         0.22844377]
 [0.22844377 1.        ]]
```
#### wiki-50000 (min-freq 1)
```
batch number: 155520.125
epoch 0, batch 155520: 7.2749
batch number: 155520.125
epoch 1, batch 155520: 6.8959
batch number: 155520.125
epoch 2, batch 155520: 6.7349
CPU times: user 11h 53min 38s, sys: 6h 1min 25s, total: 17h 55min 3s
Wall time: 17h 58min 4s

[[1.         0.15954327]
 [0.15954327 1.        ]]
```

Do you think the model performs good or bad? Why?

[3 marks]

- Training takes a long time
- looking on pearson correlation, the model performs not to good, but still finds similarities
- there are 13 words of the gold standard, which do not appear in the wiki-corpus-50000. This lowers the expressiveness of the pearson correlation

Select the 10 best and 10 worst performing word pairs, can you see any patterns that explain why *these* are the best and worst word pairs?

[3 marks]

Suggest some ways of improving the model we apply to WordSim353.

[3 marks]

- use larger corpus
- change hyperparameters (more epochs, higher learning rate, ...)
- refine model (add dropout, ...)

If we consider a scenario where we use these embeddings in a downstream task, for example sentiment analysis (roughly: determining whether a sentence is positive or negative). 

Give some examples why the sentiment analysis model would benefit from our embeddnings and one examples why our embeddings could hur the performance of the sentiment model.

[3 marks]

# Language modeling

In this second part we'll build a simple LSTM language model. Your task is to construct a model which takes a sentence as input and predict the next word for each word in the sentence. For this you'll use the ```LSTM``` class provided by PyTorch (https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html). You can read more about the LSTM here: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

NOTE!!!: Use the same dataset (wiki-corpus.50000.txt) as before.

Our setup is similar to before, we first encode the words as distributed representations then pass these to the LSTM and for each output we predict the next word.

For this we'll build a new dataloader with torchtext, the file we pass to the dataloader should contain one sentence per line, with words separated by whitespace.

```
word_1, ..., word_n
word_1, ..., word_k
...
```

in this dataloader you want to make sure that each sentence begins with a ```<start>``` token and ends with a ```<end>``` token, there is a keyword argument in ```Field``` for this :). But other than that, as before you read the dataset and output a iterator over the dataset and a vocabulary. 

Implement the dataloader, language model and the training loop (the training loop will basically be the same as for word2vec).

[12 marks]

In [75]:
# you can change these numbers to suit your needs as before :)
lm_hyperparameters = {'epochs': 3,
                      'batch_size': 1,
                      'learning_rate': 0.001,
                      'embedding_dim': 128,
                      'output_dim': 128}

In [82]:
from collections import namedtuple
from torch.utils.data import Dataset

OCCURRENCE_THRESHOLD = 1
UNKNOWN_TOKEN = '<UNK>'
START_TOKEN = '<<<'
END_TOKEN = '>>>'

class LMDataset(Dataset):
    def __init__(self, file):
        samples = []
        with open(file, encoding='UTF-8') as f:
            lines = f.readlines()[:1000]
            occurrences = {}
            for line in lines:
                tokenized = [word.lower() for word in line.rstrip('\n').split(' ')]
                for word in tokenized:
                    occurrences[word] = occurrences.get(word, 0) + 1

            unknown_words = [word for word in occurrences if occurrences[word] <= OCCURRENCE_THRESHOLD]

            for line in lines:
                tokenized = [UNKNOWN_TOKEN if word in unknown_words else word.lower() for word in line.rstrip('\n').split(' ')]
                samples.append(tokenized)


        vocab_list = list(occurrences.keys())
        vocab_list.extend([UNKNOWN_TOKEN, START_TOKEN, END_TOKEN])
        self.vocab = {word: index for index, word in enumerate(vocab_list)}
        self.data = samples

    def __getitem__(self, idx):
        sentence = self.data[idx]
        encoded_sentence = [self.vocab[word] for word in sentence]
        Sample = namedtuple('Sample', 'train eval')

        train = [self.vocab[START_TOKEN], *encoded_sentence]
        encoded_sentence.append(self.vocab[END_TOKEN])
        return Sample(train, encoded_sentence)

    def __len__(self):
        return len(self.data)

    def getVocab(self):
        return self.vocab

In [83]:
dataset = LMDataset('data/wiki-corpus.50000.txt')
dataset[1]

Sample(train=[7106, 30, 35, 36, 7105, 38, 39, 40, 41, 42, 7, 43, 10, 7, 7105, 23, 45, 39, 46, 34], eval=[30, 35, 36, 7105, 38, 39, 40, 41, 42, 7, 43, 10, 7, 7105, 23, 45, 39, 46, 34, 7107])

In [84]:
data_path = 'data/'

def get_lm_data(path, batch_size):
    dataset = LMDataset(path)
    dataloader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            collate_fn=lambda x: x)

    return dataloader

In [85]:
import torch.nn as nn

class LM_withLSTM(nn.Module):
    def __init__(self, vocab_size, dimensions, out_dim, num_layers):
        super(LM_withLSTM, self).__init__()
        self.embeddings =  nn.Embedding(vocab_size, dimensions)
        self.LSTM = nn.LSTM(dimensions, out_dim, num_layers=num_layers)
        self.predict_word = nn.Linear(out_dim, vocab_size)

    def forward(self, seq):
        embedded_seq = self.embeddings(seq)
        timestep_reprentation, *_ = self.LSTM(embedded_seq)
        predicted_words = self.predict_word(timestep_reprentation)

        return predicted_words

In [86]:
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
import torch.optim as optim

# load data
dataloader = get_lm_data(data_path + 'wiki-corpus.50000.txt', lm_hyperparameters['batch_size'])


# build model and construct loss/optimizer
lm_model = LM_withLSTM(len(dataloader.dataset.getVocab()),
                       lm_hyperparameters['embedding_dim'],
                       lm_hyperparameters['output_dim'],
                       1)
lm_model.to(device)

loss_fn = CrossEntropyLoss()
optimizer = optim.Adam(cbow_model.parameters(), lr=lm_hyperparameters['learning_rate'])

# start training loop
for epoch in range(lm_hyperparameters['epochs']):
    total_loss = 0
    print(f'batch count: {math.floor(len(dataloader.dataset) / dataloader.batch_size)}')
    for i, batch in enumerate(dataloader):
        # the strucure for each BATCH is:
        # <start>, w0, ..., wn, <end>

        # when training the model, at each input we predict the *NEXT* token
        # consequently there is nothing to predict when we give the model 
        # <end> as input. 
        # thus, we do not want to give <end> as input to the model, select 
        # from each batch all tokens except the last. 
        # tip: use pytorch indexing/slicing (same as numpy) 
        # (https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html#operations-on-tensors)
        # (https://jhui.github.io/2018/02/09/PyTorch-Basic-operations/)
        input_sentence = torch.tensor([sample.train for sample in batch])
        gold_data = torch.tensor([sample.eval for sample in batch])

        # send your batch of sentences to the model
        output = lm_model(input_sentence)

        # for each output, the model predict the NEXT token, so we have to reshape 
        # our dataset again. On timestep t, we evaluate on token t+1. That is,
        # we never predict the <start> token ;) so this time, we select all but the first 
        # token from sentences (that is, all the tokens that we predict)

        # the shape of the output and sentence variable need to be changed,
        # for the loss function. Details are in the documentation.
        # You can use .view(...,...) to reshape the tensors  
        loss = loss_fn(output.view(-1, len(dataloader.dataset.getVocab())), gold_data.view(-1))
        total_loss += loss.item()

        # print average loss for the epoch
        print(f'epoch {epoch},', f'batch {i}:', np.round(total_loss / (i + 1), 4), end='\r')

        # compute gradients
        loss.backward()

        # update parameters
        optimizer.step()

        # reset gradients
        optimizer.zero_grad()
    print()

batch count: 1000
epoch 0, batch 999: 8.8792
batch count: 1000
atch 621: 8.8787epoch 1, batch 646: 8.8788epoch 1, batch 667: 8.879epoch 1, batch 684: 8.8791epoch 1, batch 705: 8.8792epoch 1, batch 727: 8.879epoch 1, batch 773: 8.8789epoch 1, batch 803: 8.8787epoch 1, batch 828: 8.8789epoch 1, batch 850: 8.8791epoch 1, batch 870: 8.8792epoch 1, batch 892: 8.8792epoch 1, batch 915: 8.879epoch 1, batch 937: 8.8789epoch 1, batch 961: 8.8789epoch 1, batch 990: 8.8792
batch count: 1000
poch 2, batch 222: 8.8801epoch 2, batch 251: 8.8801epoch 2, batch 278: 8.8799epoch 2, batch 309: 8.8796epoch 2, batch 338: 8.8802epoch 2, batch 364: 8.8809epoch 2, batch 395: 8.8803epoch 2, batch 422: 8.8797epoch 2, batch 451: 8.8799epoch 2, batch 475: 8.88epoch 2, batch 503: 8.8796epoch 2, batch 533: 8.8796epoch 2, batch 563: 8.8793epoch 2, batch 593: 8.8792epoch 2, batch 624: 8.8789epoch 2, batch 656: 8.879epoch 2, batch 687: 8.8791epoch 2, batch 716: 8.8791epoch 2, batch 745: 8.8793epoch 2, batch 777: 8.879

### Evaluating the language model

We'll evaluate our model using the BLiMP dataset (https://github.com/alexwarstadt/blimp). The BLiMP dataset contains sets of linguistic minimal pairs for various syntactic and semantic phenomena, We'll evaluate our model on *existential quantifiers* (link: https://github.com/alexwarstadt/blimp/blob/master/data/existential_there_quantifiers_1.jsonl). This data, as the name suggests, investigate whether language models assign higher probability to *correct* usage of there-quantifiers. 

An example entry in the dataset is: 

```
{"sentence_good": "There was a documentary about music irritating Allison.", "sentence_bad": "There was each documentary about music irritating Allison.", "field": "semantics", "linguistics_term": "quantifiers", "UID": "existential_there_quantifiers_1", "simple_LM_method": true, "one_prefix_method": false, "two_prefix_method": false, "lexically_identical": false, "pairID": "0"}
```

Download the dataset and build a datareader (similar to what you did for word2vec). The dataset structure you should aim for is (you don't need to worry about the other keys for this assignment):

```
good_sentence_1, bad_sentence_1
...
```

your task now is to compare the probability assigned to the good sentence with to the probability assigned to the bad sentence. To compute a probability for a sentence we consider the product of the probabilities assigned to the *gold* tokens, remember, at timestep ```t``` we're predicting which token comes *next* e.g. ```t+1``` (basically, you do the same thing as you did when training).

In rough pseudo code what your code should do is:

```
accuracy = []
for good_sentence, bad_sentence in dataset:
    gs_lm_output = LanguageModel(good_sentence)
    gs_token_probabilities = softmax(gs_lm_output)
    gs_sentence_probability = product(gs_token_probabilities[GOLD_TOKENS])

    bs_lm_output = LanguageModel(bad_sentence)
    bs_token_probabilities = softmax(bs_lm_output)
    bs_sentence_probability = product(bs_token_probabilities[GOLD_TOKENS])

    # int(True) = 1 and int(False) = 0
    is_correct = int(gs_sentence_probability > bs_sentence_probability)
    accuracy.append(is_correct)

print(numpy.mean(accuracy))
    
```

[6 marks]

In [87]:
# your code goes here
import json


def evaluate_model(path, vocab, model):
    accuracy = []
    with open(path) as f:
        # iterate over one pair of sentences at a time
        for line in f:
            # load the data
            data = json.loads(line)
            good_s = data['sentence_good']
            bad_s = data['sentence_bad']

            # the data is tokenized as whitespace
            tok_good_s = ...
            tok_bad_s = ...

            # encode your words as integers using the vocab from the dataloader, size is (S)
            # we use unsqueeze to create the batch dimension 
            # in this case our input is only ONE batch, so the size of the tensor becomes: 
            # (S) -> (1, S) as the model expects batches
            enc_good_s = torch.tensor([_ for x in tok_good_s], device=device).unsqueeze(0)
            enc_bad_s = torch.tensor([_ for x in tok_bad_s], device=device).unsqueeze(0)

            # pass your encoded sentences to the model and predict the next tokens
            good_s = LM_withLSTM(enc_good_s)
            bad_s = LM_withLSTM(enc_bad_s)

            # get probabilities with softmax
            gs_probs = F.softmax(...)
            bs_probs = F.softmax(...)

            # select the probability of the gold tokens
            gs_sent_prob = find_token_probs(gs_probs, enc_good_s)
            bs_sent_prob = find_token_probs(bs_probs, enc_bad_s)

            accuracy.append(int(gs_sent_prob > bs_sent_prob))

    return accuracy


def find_token_probs(model_probs, encoded_sentece):
    probs = []

    # iterate over the tokens in your encoded sentence
    for token, gold_token in enumerate(encoded_sentece):
        # select the probability of the gold tokens and save
        # hint: pytorch indexing is helpful here ;)
        prob = ...
        probs.append(prob)
    sentence_prob = ...
    return sentence_prob


path = 'existential_there_quantifiers_1.jsonl'
accuracy = evaluate_model(path, ..., ...)

print('Final accuracy:')
print(np.round(np.mean(accuracy), 3))


FileNotFoundError: [Errno 2] No such file or directory: 'existential_there_quantifiers_1.jsonl'

### Analysis

Our model get some score, say, 55% correct predictions. Is this good? Suggest some *baseline* (i.e. a stupid "model" we hope ours is better than) we can compare the model against.

[3 marks]

Suggest some improvements you could make to your language model.

[3 marks]

Suggest some other metrics we can use to evaluate our system

[2 marks]

# Literature


Neural architectures:
* Y. Bengio, R. Ducharme, P. Vincent, and C. Janvin. A neural probabilistic language model. (Links to an external site.) Journal of Machine Learning Research, 3(6):1137–1155, 2003. (Sections 3 and 4 are less relevant today and hence you can glance through them quickly. Instead, look at the Mikolov papers where they describe training word embeddings with the current neural network architectures.)
* T. Mikolov, K. Chen, G. Corrado, and J. Dean. Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781, 2013.
* T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.
    


## Statement of contribution

Briefly state how many times you have met for discussions, who was present, to what degree each member contributed to the discussion and the final answers you are submitting.

Total marks: 63